# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


from api_keys import g_key
from pprint import pprint


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path=os.path.join('..','WeatherPy','city_data.csv')
df=pd.read_csv(path)
df1=df.drop(['Unnamed: 0'], axis=1)
df1


,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp (f),Wind Speed (mph)
0,cayenne,92,NaN,1583761906,73,6.42,-48.99,79.052,23.26
1,ushuaia,100,NaN,1583761906,96,-84.57,-40.33,-17.428,11.63
2,port alfred,100,NaN,1583761906,95,-87.25,40.27,-44.788,13.89
3,jamestown,75,US,1583761906,85,46.61,-99.28,12.200,10.29
4,cabo san lucas,0,NaN,1583761906,79,11.28,-114.96,80.852,16.89
...,...,...,...,...,...,...,...,...,...
611,brae,98,NaN,1583761960,90,68.36,125.98,-22.648,3.80
612,sept-iles,40,NaN,1583761960,78,-16.62,-124.52,79.592,3.60
613,dandong,0,SA,1583761960,22,28.75,41.47,70.412,6.11
614,slave lake,100,NaN,1583761960,62,53.85,-178.36,31.712,19.15


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations=df1[['Lat','Lon']]
rating=df1['Humidity']
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations, weights=rating, dissipating=False, max_intensity=max(rating),point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
df2=df1.loc[(df1['Max Temp (f)']>70)&(df1['Max Temp (f)']<80)]
df3=df2.loc[df1['Wind Speed (mph)']<10]
df4=df3.loc[df1['Cloudiness']==0]
df5=df4.dropna(how='any')

locations=df5[['Lat','Lon']]
rating=df5['Humidity']
fig=gmaps.figure()

heat_layer=gmaps.heatmap_layer(locations, weights=rating, dissipating=False, max_intensity=max(rating),point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
hotel_df=df5
hotel_df['Hotel Name']=""
hotel_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lon=row['Lon']
    location=lat,lon
    city=row['City']
    
    params = {"location": f"{lat},{lon}",
    "type": "lodging",
    "radius": 50000,
    "key": g_key}
   
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")

    response = requests.get(base_url, params).json()
#     pprint(response)
    
#     extract results
    results = response['results']
#     pprint(results)
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
# #         types_df.loc[index, 'name'] = results[0]['name']
# #         types_df.loc[index, 'address'] = results[0]['vicinity']
# #         types_df.loc[index, 'price_level'] = results[0]['price_level']
# #         types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

c:\users\belac\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Retrieving Results for Index 437: urumqi.
Closest hotel is 海航产权酒店.
------------
Retrieving Results for Index 571: karkaralinsk.
Closest hotel is Hôtel Les Amandiers.
------------
Retrieving Results for Index 613: dandong.
Closest hotel is مزرعه كامل موسى.
------------


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map